# 단일 PDF 파일 챗봇 샘플 코드

### FAISS 활용


### 환경설정

In [ ]:
# 패키지 설치 후 커널 리스타트 해주세요.

!pip install boto3 langchain langchain_aws langchain_community -U

In [1]:
import os
import boto3
import time
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_aws import ChatBedrock

# Bedrock 클라이언트 설정
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"  # 사용하는 리전으로 변경하세요
)


In [2]:
%%time

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader("./rag_data/Steel Business Briefing_15 Jul 2024.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Bedrock 임베딩 설정
embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v1" # Titan Embeddings V1
)

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(texts, embeddings)

# Claude 3 Sonnet LLM 설정
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 4096, 
        "temperature": 0.5
    }
)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# 쿼리 실행 함수
def query_pdf(question):
    response = qa_chain.run(question)
    return response


CPU times: user 1.65 s, sys: 517 ms, total: 2.17 s
Wall time: 17.9 s


In [3]:
%%time

# 사용자 쿼리
question = """
PDF 문서의 주요 내용은 무엇인가요?
"""

answer = query_pdf(question)
print(f"질문: {question}")
print(f"답변: {answer}")
print()

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


질문: 
PDF 문서의 주요 내용은 무엇인가요?

답변: 이 PDF 문서는 S&P Global Commodity Insights가 제공하는 2024년 7월 15일자 철강 관련 일일 브리핑 보고서인 것 같습니다. 주요 내용은 다음과 같습니다:

- 중국 정부의 정책 기대감으로 중국 열연코일 가격이 상승했다는 내용이 있습니다.

- 클리브랜드-클리프스 철강회사가 U.S. 스틸을 인수하려 했지만, 결국 일본 니폰제철에 인수되었다는 내용이 있습니다. 

- 남아프리카 대통령이 EU의 탄소국경조정메커니즘(CBAM) 대비를 위해 산업계의 탄소집약도를 낮출 필요가 있다고 언급한 내용이 있습니다.

- 유럽 열연코일 가격 평가 근거와 관련 데이터 코드도 포함되어 있습니다.

- 마지막에는 S&P 데이터 사용과 관련된 주의사항이 적혀 있습니다.

전반적으로 해당일의 글로벌 철강 가격 동향과 주요 이슈 등을 다루고 있는 산업 전문 브리핑 보고서인 것 같습니다.

CPU times: user 26.1 ms, sys: 1.42 ms, total: 27.5 ms
Wall time: 10.4 s
